In [23]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# MLflow setup
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('SARIMAX_Example_2')

# Load data
df = pd.read_parquet('../data/sequences.parquet')
df['TIME'] = pd.to_datetime(df['TIME'])
df.set_index('TIME', inplace=True)

# Parameters
SEQUENCE_ID = 1  # Example with one sequence
TRAIN_SIZE = 168  # 7 days
VAL_SIZE = 48    # 2 days
TEST_SIZE = 24   # 1 day

# Cell 2: Define SARIMAX configurations
model_configs = [
    {
        'name': 'sarimax_complex',
        'order': (3, 1, 3),  # Higher ARIMA order (p, d, q)
        'seasonal_order': (2, 1, 2, 24),  # Higher seasonal order (P, D, Q, S) with period 24
        'trend': 'c',  # Constant trend
        'exog': ['PV1_Voltage', 'PV1_Current']  # Exogenous variables
    }
]

# Cell 3: Preprocessing function
def preprocess_data(data):
    """Preprocess sequence data"""
    # Remove outliers
    Q1 = data['Power'].quantile(0.25)
    Q3 = data['Power'].quantile(0.75)
    IQR = Q3 - Q1
    data = data[
        (data['Power'] >= Q1 - 1.5 * IQR) &
        (data['Power'] <= Q3 + 1.5 * IQR)
    ].copy()
    
    # Scale data
    scaler = StandardScaler()
    data['Power_scaled'] = scaler.fit_transform(data[['Power']])
    
    return data, scaler

# Get sequence data
sequence_data = df[df['sequence'] == SEQUENCE_ID].copy()
sequence_data, scaler = preprocess_data(sequence_data)

# Use only half of the dataset
sequence_data = sequence_data.iloc[:len(sequence_data)//2]



In [27]:
# Cell 4: Training and evaluation with SARIMAX

# Print total dataset size
print(f"Total dataset size: {len(sequence_data)}")

# Adjust split sizes
TRAIN_SIZE = 100  # Adjusted to fit the dataset
VAL_SIZE = 28     # Adjusted to fit the dataset
TEST_SIZE = 28    # Adjusted to fit the dataset

for config in model_configs:
    # Start MLflow run
    with mlflow.start_run(run_name=f"sequence_{SEQUENCE_ID}_{config['name']}"):
        try:
            # Log parameters
            mlflow.log_params({
                'sequence': SEQUENCE_ID,
                'model_name': config['name'],
                'order_p': config['order'][0],
                'order_d': config['order'][1],
                'order_q': config['order'][2],
                'seasonal_P': config['seasonal_order'][0],
                'seasonal_D': config['seasonal_order'][1],
                'seasonal_Q': config['seasonal_order'][2],
                'seasonal_period': config['seasonal_order'][3],
                'trend': config['trend'],
                'exog': config['exog'],
                'train_size': TRAIN_SIZE,
                'val_size': VAL_SIZE,
                'test_size': TEST_SIZE
            })
            
            # Split data
            train_data = sequence_data['Power_scaled'][:TRAIN_SIZE]
            val_data = sequence_data['Power'][TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE]
            test_data = sequence_data['Power'][TRAIN_SIZE+VAL_SIZE:TRAIN_SIZE+VAL_SIZE+TEST_SIZE]
            
            # Debug: Print dataset lengths
            print(f"\nDebugging for {config['name']}:")
            print(f"Training data length: {len(train_data)}")
            print(f"Validation data length: {len(val_data)}")
            print(f"Test data length: {len(test_data)}")
            
            # Prepare exogenous variables (if any)
            if config['exog'] is not None:
                exog_train = sequence_data[config['exog']][:TRAIN_SIZE]
                exog_val = sequence_data[config['exog']][TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE]
                exog_test = sequence_data[config['exog']][TRAIN_SIZE+VAL_SIZE:TRAIN_SIZE+VAL_SIZE+TEST_SIZE]
                
                # Combine validation and test exogenous variables
                exog_full = pd.concat([exog_val, exog_test], axis=0)
            else:
                exog_train = exog_full = None
            
            # Train SARIMAX model
            print(f"Training model {config['name']}...")
            model = SARIMAX(
                train_data,
                exog=exog_train,
                order=config['order'],
                seasonal_order=config['seasonal_order'],
                trend=config['trend'],
                enforce_stationarity=False
            )
            
            # Fit model
            fitted_model = model.fit(disp=False, method='powell', maxiter=50)  # Increased maxiter for better convergence
            print("Model training completed.")
            
            # Make predictions
            print("Making predictions...")
            
            # Validation predictions
            val_start = len(train_data)
            val_end = val_start + len(val_data) - 1
            
            # Debug: Print prediction range
            print(f"Validation prediction range: start={val_start}, end={val_end}")
            
            if val_end >= val_start:
                # Predict for the full range (validation + test)
                full_predictions = fitted_model.predict(
                    start=val_start,
                    end=val_end + len(test_data),
                    exog=exog_full,
                    dynamic=False
                )
                
                # Split predictions into validation and test
                val_predictions = full_predictions[:len(val_data)]
                test_predictions = full_predictions[len(val_data):]
                
                # Inverse transform predictions
                val_predictions = scaler.inverse_transform(val_predictions.values.reshape(-1, 1)).ravel()
                test_predictions = scaler.inverse_transform(test_predictions.values.reshape(-1, 1)).ravel()
            else:
                raise ValueError(f"Validation prediction range is invalid: start={val_start}, end={val_end}")
            
            print("Predictions completed.")
            
            # Calculate metrics
            val_rmse = np.sqrt(mean_squared_error(val_data, val_predictions))
            val_mae = mean_absolute_error(val_data, val_predictions)
            test_rmse = np.sqrt(mean_squared_error(test_data, test_predictions))
            test_mae = mean_absolute_error(test_data, test_predictions)
            
            # Log metrics to MLflow
            print("Logging metrics...")
            mlflow.log_metrics({
                'val_rmse': val_rmse,
                'val_mae': val_mae,
                'test_rmse': test_rmse,
                'test_mae': test_mae
            })
            print("Metrics logged.")
            
            # Print results
            print(f"\nResults for {config['name']}:")
            print(f"Validation RMSE: {val_rmse:.4f}")
            print(f"Validation MAE: {val_mae:.4f}")
            print(f"Test RMSE: {test_rmse:.4f}")
            print(f"Test MAE: {test_mae:.4f}")
            
        except Exception as e:
            print(f"Error in {config['name']}: {str(e)}")
            continue

Total dataset size: 156

Debugging for sarimax_simple:
Training data length: 100
Validation data length: 28
Test data length: 28
Training model sarimax_simple...
Model training completed.
Making predictions...
Validation prediction range: start=100, end=127
Predictions completed.
Logging metrics...
Metrics logged.

Results for sarimax_simple:
Validation RMSE: 0.1239
Validation MAE: 0.0930
Test RMSE: 0.2737
Test MAE: 0.2696
🏃 View run sequence_1_sarimax_simple at: http://localhost:5000/#/experiments/6/runs/20cb44ce013e4243a65097e71ea10a95
🧪 View experiment at: http://localhost:5000/#/experiments/6

Debugging for sarimax_with_trend:
Training data length: 100
Validation data length: 28
Test data length: 28
Training model sarimax_with_trend...
Model training completed.
Making predictions...
Validation prediction range: start=100, end=127
Predictions completed.
Logging metrics...
Metrics logged.

Results for sarimax_with_trend:
Validation RMSE: 0.1736
Validation MAE: 0.1629
Test RMSE: 0.212

In [25]:
df.head(20)

PV1_Voltage  PV1_Current  PV2_Voltage  PV2_Current  \
TIME                                                                      
2023-01-01 08:22:00        499.6         0.04            0          0.0   
2023-01-01 08:24:00        499.6         0.04            0          0.0   
2023-01-01 08:26:00        499.6         0.04            0          0.0   
2023-01-01 08:28:00        499.6         0.04            0          0.0   
2023-01-01 08:30:00        499.6         0.04            0          0.0   
2023-01-01 08:32:00        499.6         0.04            0          0.0   
2023-01-01 08:34:00        499.6         0.04            0          0.0   
2023-01-01 08:36:00        499.6         0.04            0          0.0   
2023-01-01 08:38:00        499.6         0.04            0          0.0   
2023-01-01 08:40:00        499.6         0.04            0          0.0   
2023-01-01 08:42:00        565.5         0.07            0          0.0   
2023-01-01 08:44:00        565.5         0.07            0          0.0   
2023-01-01 08:46:00        625.5         0.10            0          0.0   
2023-01-01 08:48:00        625.5         0.10            0          0.0   
2023-01-01 08:50:00        625.5         0.10            0          0.0   
2023-01-01 08:52:00        509.0         0.45            0          0.0   
2023-01-01 08:54:00        509.0         0.45            0          0.0   
2023-01-01 08:56:00        557.4         0.54            0          0.0   
2023-01-01 08:58:00        557.4         0.54            0          0.0   
2023-01-01 09:00:00        557.4         0.54            0          0.0   

                     PV3_Voltage  InputPower  Power  sequence  
TIME                                                           
2023-01-01 08:22:00            0       0.000  2.385         1  
2023-01-01 08:24:00            0       0.000  2.395         1  
2023-01-01 08:26:00            0       0.000  2.394         1  
2023-01-01 08:28:00            0       0.000  2.411         1  
2023-01-01 08:30:00            0       0.000  2.483         1  
2023-01-01 08:32:00            0       0.000  2.479         1  
2023-01-01 08:34:00            0       0.000  2.486         1  
2023-01-01 08:36:00            0       0.000  2.493         1  
2023-01-01 08:38:00            0       0.000  2.478         1  
2023-01-01 08:40:00            0       0.000  2.468         1  
2023-01-01 08:42:00            0       0.005  2.479         1  
2023-01-01 08:44:00            0       0.005  2.414         1  
2023-01-01 08:46:00            0       0.034  2.388         1  
2023-01-01 08:48:00            0       0.034  2.433         1  
2023-01-01 08:50:00            0       0.034  2.472         1  
2023-01-01 08:52:00            0       0.223  2.456         1  
2023-01-01 08:54:00            0       0.223  2.454         1  
2023-01-01 08:56:00            0       0.282  2.454         1  
2023-01-01 08:58:00            0       0.282  2.394         1  
2023-01-01 09:00:00            0       0.282  2.377         1

In [26]:
# Define SARIMAX configurations
model_configs = [
    {
        'name': 'sarimax_simple',
        'order': (1, 0, 1),  # ARIMA order (p, d, q)
        'seasonal_order': (1, 0, 1, 12),  # Seasonal order (P, D, Q, S) with period 12
        'trend': 'n',  # No trend
        'exog': None  # No exogenous variables
    },
    {
        'name': 'sarimax_with_trend',
        'order': (1, 0, 1),  # ARIMA order (p, d, q)
        'seasonal_order': (1, 0, 1, 12),  # Seasonal order (P, D, Q, S) with period 12
        'trend': 'c',  # Constant trend
        'exog': None  # No exogenous variables
    },
    {
        'name': 'sarimax_with_exog',
        'order': (1, 0, 1),  # ARIMA order (p, d, q)
        'seasonal_order': (1, 0, 1, 12),  # Seasonal order (P, D, Q, S) with period 12
        'trend': 'n',  # No trend
        'exog': ['PV1_Voltage', 'PV1_Current']  # Example exogenous variables
    }
]

In [8]:
def preprocess_data(data):
    """Preprocess sequence data"""
    # Remove outliers
    Q1 = data['Power'].quantile(0.25)
    Q3 = data['Power'].quantile(0.75)
    IQR = Q3 - Q1
    data = data[
        (data['Power'] >= Q1 - 1.5 * IQR) &
        (data['Power'] <= Q3 + 1.5 * IQR)
    ].copy()
    
    # Scale data
    scaler = StandardScaler()
    data['Power_scaled'] = scaler.fit_transform(data[['Power']])
    
    return data, scaler

# Get sequence data
sequence_data = df[df['sequence'] == SEQUENCE_ID].copy()
sequence_data, scaler = preprocess_data(sequence_data)

# Use only half of the dataset
sequence_data = sequence_data.iloc[:len(sequence_data)//2]

In [9]:
best_metrics = {'val_rmse': float('inf')}
best_config = None

# Grid Search over hyperparameters
for order in param_grid['order']:
    for seasonal_order in param_grid['seasonal_order']:
        for trend in param_grid['trend']:
            config = {
                'order': order,
                'seasonal_order': seasonal_order,
                'trend': trend
            }
            
            # Start MLflow run
            with mlflow.start_run(run_name=f"sequence_{SEQUENCE_ID}_tuning"):
                try:
                    # Log parameters
                    mlflow.log_params({
                        'sequence': SEQUENCE_ID,
                        'order_p': order[0],
                        'order_d': order[1],
                        'order_q': order[2],
                        'seasonal_P': seasonal_order[0],
                        'seasonal_D': seasonal_order[1],
                        'seasonal_Q': seasonal_order[2],
                        'seasonal_period': seasonal_order[3],
                        'trend': trend
                    })
                    
                    # Split data
                    train_data = sequence_data['Power_scaled'][:TRAIN_SIZE]
                    val_data = sequence_data['Power'][TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE]
                    
                    # Train model
                    model = SARIMAX(
                        train_data,
                        order=order,
                        seasonal_order=seasonal_order,
                        trend=trend,
                        enforce_stationarity=False,
                        initialization='approximate'
                    )
                    
                    # Fit with limited iterations
                    fitted_model = model.fit(disp=False, method='powell', maxiter=50)
                    
                    # Make predictions
                    val_predictions = fitted_model.predict(
                        start=len(train_data),
                        end=len(train_data) + len(val_data) - 1,
                        dynamic=False
                    )
                    val_predictions = scaler.inverse_transform(val_predictions.reshape(-1, 1)).ravel()
                    
                    # Calculate metrics
                    val_rmse = np.sqrt(mean_squared_error(val_data, val_predictions))
                    val_mae = mean_absolute_error(val_data, val_predictions)
                    
                    # Log metrics
                    mlflow.log_metrics({
                        'val_rmse': val_rmse,
                        'val_mae': val_mae
                    })
                    
                    # Update best configuration
                    if val_rmse < best_metrics['val_rmse']:
                        best_metrics = {
                            'val_rmse': val_rmse,
                            'val_mae': val_mae
                        }
                        best_config = config
                    
                except Exception as e:
                    print(f"Error in config {config}: {str(e)}")
                    continue

# Log best configuration
print(f"Best Configuration: {best_config}")
print(f"Best Validation RMSE: {best_metrics['val_rmse']:.4f}")

Error in config {'order': (1, 0, 0), 'seasonal_order': (0, 0, 0, 0), 'trend': 'n'}: Invalid state space initialization method.
🏃 View run sequence_1_tuning at: http://localhost:5000/#/experiments/3/runs/8f6b02584554470f811bb56bfc99398b
🧪 View experiment at: http://localhost:5000/#/experiments/3
Error in config {'order': (1, 0, 0), 'seasonal_order': (0, 0, 0, 0), 'trend': 'c'}: Invalid state space initialization method.
🏃 View run sequence_1_tuning at: http://localhost:5000/#/experiments/3/runs/f03c4805952f439aa8cff80796f58ba3
🧪 View experiment at: http://localhost:5000/#/experiments/3
Error in config {'order': (1, 0, 0), 'seasonal_order': (1, 0, 1, 24), 'trend': 'n'}: Invalid state space initialization method.
🏃 View run sequence_1_tuning at: http://localhost:5000/#/experiments/3/runs/6ecf1b8b92f14b0e81410ab7ec9fa395
🧪 View experiment at: http://localhost:5000/#/experiments/3
Error in config {'order': (1, 0, 0), 'seasonal_order': (1, 0, 1, 24), 'trend': 'c'}: Invalid state space initia